In [1]:
import numpy as np
# %load_ext autoreload
# %autoreload 2
# %autosave 180

# nerf_pc = np.load("lidar_nerf_demo/generated_pc_at_start.npy")
# real_pc = np.load("lidar_nerf_demo/real_pc_at_start.npy")
# nerf_pc = np.random.randn(1000,3)
# real_pc = np.random.randn(1000,3)
# nerf_pc = np.load("generatedPointClouds/7750NeRF.npy")
# real_pc = np.load("generatedPointClouds/7750real.npy")
nerf_pc = np.load("generatedPointClouds/7800NeRF.npy")
real_pc = np.load("generatedPointClouds/7800real.npy")
# nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput.npy")
# real_pc = np.load("generatedPointClouds/MaiCityReal.npy")

# nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_frame_10.npy")
# real_pc = np.load("generatedPointClouds/MaiCityReal_frame_10.npy")

# #remove ground plane
# real_pc = real_pc[real_pc[:,2] > -1]
# nerf_pc = nerf_pc[nerf_pc[:,2] > -1]

# #debug: create simple pc
# nerf_pc = np.array([[0,0,0], 
#                     [1,1,1]])
# real_pc = np.array([[0,0,0], 
#                     [0,0,0]])

#crop real PC down the same as NeRF output so bidirectional CD doesn't punish non-overlaps
real_pc = real_pc[len(real_pc)//8:7*len(real_pc)//8,:]
# real_pc = real_pc[len(real_pc)//8:29*len(real_pc)//32,:]


#remove points too close to origin
d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
good  = np.argwhere(d > 1)[:,0]
nerf_pc = nerf_pc[good]

#remove points out of range of sensor (only needed for Mai City)
d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
good = np.argwhere(d < 100)[:,0]
nerf_pc = nerf_pc[good]

# nerf_pc = nerf_pc[::3,:]
# real_pc = real_pc[::3,:]
# nerf_pc = nerf_pc[:len(nerf_pc)//4,:]
# real_pc = real_pc[:len(real_pc)//4,:]

print(np.shape(nerf_pc))
print(np.shape(real_pc))

(47725, 3)
(49152, 3)


In [ ]:
import torch
from chamferdist import ChamferDistance

real_pc = torch.tensor(real_pc)
nerf_pc = torch.tensor(nerf_pc)

if len(real_pc.shape) == 2:
    real_pc = real_pc.unsqueeze(0)  # Add batch dimension
if len(nerf_pc.shape) == 2:
    nerf_pc = nerf_pc.unsqueeze(0)  # Add batch dimension
    
# Convert to float
real_pc = real_pc.float()
nerf_pc = nerf_pc.float()
print(nerf_pc.shape)

chamferDist = ChamferDistance()
# dist_forward = chamferDist(nerf_pc, real_pc)
# print(dist_forward)
# print(dist_forward/ nerf_pc.shape[1])
# dist_bidirectional = chamferDist(nerf_pc, real_pc, bidirectional=True)
# print(dist_bidirectional)
# print(dist_bidirectional/ nerf_pc.shape[1])

#TEST -- modify chamfer.py to produce output for calculating F-scores
# dist_forward = chamferDist(nerf_pc, real_pc, reduction = 'fscore')
dist_forward = chamferDist(nerf_pc, real_pc, reduction = 'fscore', reverse = True)
# dist_forward is actually a SQUARED distance, so I need to take the sqrt first
print(dist_forward)
dist_forward = torch.sqrt(dist_forward)
print(dist_forward)

bad = np.argwhere(dist_forward > 0.1)

print(np.shape(dist_forward)[1])
print(np.shape(bad)[1])

print(1 - np.shape(bad)[1] / np.shape(dist_forward)[1])

In [ ]:
a = .93
b = 0.93
f = 2*(a*b) / (a + b)

print(f)

In [4]:
# print(savepc)
# print(pc1_aligned)
# print(dist_forward/ np.shape(nerf_pc)[1])

from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget

plt = Plotter(N = 1, axes = 0, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

#crop real PC down the same as NeRF output so bidirectional CD doesn't punish non-overlaps
# test = real_pc[0,len(real_pc[0,:,:])//8:7*len(real_pc[0,:,:])//8,:]
# print(np.shape(test))
# print(len(real_pc[0,:,:])//8)
# disp.append(Points(test, c = "red", r = 5))

disp.append(Points(real_pc[0,:,:], c = "red", r = 5))
disp.append(Points(nerf_pc[0,:,:], c = "blue", r = 5))

plt.show(disp, "testing CD")
ViewInteractiveWidget(plt.window)


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

# calculate chamfer distance between NeRF output true distance for 25 frames Mai City Dataset

In [ ]:
CD_hist = []
chamferDist = ChamferDistance()

for frame_idx in range(25):
    print(frame_idx)
    nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_frame_" + str(frame_idx) + ".npy")
    real_pc = np.load("generatedPointClouds/MaiCityReal_frame_" + str(frame_idx) + ".npy")

    #remove points too close to origin
    d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
    good  = np.argwhere(d > 1)[:,0]
    nerf_pc = nerf_pc[good]

    #remove points out of range of sensor (only needed for Mai City)
    d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
    good = np.argwhere(d < 100)[:,0]
    nerf_pc = nerf_pc[good]
    
    print(np.shape(nerf_pc))
#     nerf_pc = nerf_pc[::2,:]
#     real_pc = real_pc[::2,:]
#     print(np.shape(nerf_pc))

    real_pc = torch.tensor(real_pc)
    nerf_pc = torch.tensor(nerf_pc)

    if len(real_pc.shape) == 2:
        real_pc = real_pc.unsqueeze(0)  # Add batch dimension
    if len(nerf_pc.shape) == 2:
        nerf_pc = nerf_pc.unsqueeze(0)  # Add batch dimension

    # Convert to float
    real_pc = real_pc.float()
    nerf_pc = nerf_pc.float()
    
    dist_bidirectional = chamferDist(nerf_pc, real_pc, bidirectional=True)
    dist_bidirectional /= nerf_pc.shape[1]
    print(dist_bidirectional)
    
    CD_hist = np.append(CD_hist, dist_bidirectional)

In [ ]:
from matplotlib import pyplot as p

CD_hist_2 = CD_hist.copy()

fig, ax = p.subplots()
ax.set_title("Chamfer Distance for Mai City Dataset")
# ax.plot(CD_hist_15, label = "sample every 15th")
ax.plot(CD_hist_10, label = "sample every 10th")
# ax.plot(CD_hist_5, label = "sample every 5th")
ax.plot(CD_hist_2, label = "sample every 2nd")
ax.plot(CD_hist, label = "every point")
ax.set_xlabel("test frame index")
ax.set_ylabel("Chamfer Distance (cm)")
ax.legend(loc="best")

In [ ]:
# print(np.mean(CD_hist))
# np.save("generatedPointClouds/MaiCity_CD_hist.npy", CD_hist)

In [58]:
#combine all NeRF outputs to single point cloud to render with open3d
combined_pc = np.zeros([0,3])
for frame_idx in range(50): #25
    print(frame_idx)
#     nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_frame_" + str(frame_idx) + ".npy") #
    nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_velodyne_format_frame_" + str(frame_idx) + ".npy") #OS1 projections
#     nerf_pc = np.load("generatedPointClouds/MaiCityReal_frame_" + str(frame_idx) + ".npy") #DEBUG -- true scans

    #remove near zero points
#     nerf_pc = nerf_pc*200
    dists = np.sqrt(nerf_pc[:,0]**2 + nerf_pc[:,1]**2 + nerf_pc[:,2]**2)
    good = np.argwhere(dists > 2)[:,0]
#     print(good)
    nerf_pc = nerf_pc[good]

    nerf_pc = nerf_pc + np.array([frame_idx*.5,0.,0.])
#     nerf_pc = nerf_pc + np.array([frame_idx*1.,0.,0.])
    
    combined_pc = np.append(combined_pc, nerf_pc, axis = 0)

# print(np.shape(combined_pc))

# #crop to help with marching cubes??
# combined_pc = combined_pc[abs(combined_pc[:,0]) < 20]
# combined_pc = combined_pc[abs(combined_pc[:,1]) < 20]
# np.save("generatedPointClouds/CombinedMaiCity.npy", combined_pc)

import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(combined_pc)
o3d.io.write_point_cloud("MaiCityNeRF.ply", pcd)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


True

In [42]:
#TEST -- load and crop HD Map to test marching cubes
import trimesh
pl = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-1stSection - mesh.ply'
raw_points = trimesh.load(pl).vertices
# points[:,2] = -points[:,2] #flip z
# points = points[::10,:] #downsample
points = raw_points.copy()

points = points[points[:,0]>0]
points = points[points[:,1]> - 2]

combined_pc = points
# np.save("generatedPointClouds/NCTestMap.npy", combined_pc)

import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(combined_pc)
o3d.io.write_point_cloud("NCTestMap.ply", pcd)


True

In [41]:
plt = Plotter(N = 1, axes = 0, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

disp.append(Points(combined_pc, c = "red", r = 2.5, alpha = 0.3))
disp.append(Points(raw_points, c = "black", r = 2.5, alpha = 0.1))

plt.show(disp, "testing CD")
ViewInteractiveWidget(plt.window)


ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

### Load 100 raw frames from Mai City sequence 01 and combine to a single point cloud

In [57]:
# test = np.loadtxt("/media/derm/06EF-127D4/mai_city/txt/01/000000")

combined_pc = np.zeros([0,3])
for idx1 in range(100):
    print(idx1)
    pcfn = "/media/derm/06EF-127D4/mai_city/txt/01/" +  f"{idx1:06}"
    
    pc = np.loadtxt(pcfn)
    pc += np.array([1.*idx1, 0.,0.])
    pc = pc[::2]
    
    combined_pc = np.append(combined_pc, pc, axis = 0)

print(combined_pc)
import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(combined_pc)
o3d.io.write_point_cloud("MaiCityGTMap.ply", pcd)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
[[ 98.11171    0.        -1.729996]
 [ 58.456245   0.        -1.729998]
 [ 41.457108   0.        -1.729998]
 ...
 [103.211807   0.025409  -1.729996]
 [103.011147   0.024199  -1.729996]
 [102.826103   0.023082  -1.729996]]


True